# Introduction to Business Problem #

Opening a new Japanese Restaurant in Bangkok, Thailand

The objective of this report is to determine the best possible location to open a japanese restaurant in Bangkok, Thailand based on the different localities of the city, already established Italian restaurant in varios geographical location and ease of accessibility by maximum number of people so that the revenue from the latest venture can be maximized.


# Data #

This project will use data from :

Geopy - For getting the co-ordinated of different locations.
Foursquare API - To get the list of vanues and their details around a given location.